In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [226]:
def lookup_boardgame(ids):    
    '''
    converts boardgame ids into boardgame names
    '''
    return boardgames.loc[ids, 'name'].tolist()

def lookup_user_id(user_name):
    '''
    returns the user id of a user
    '''
    user_id = users[users['user_name']==user_name]['user_id'].tolist()[0]
    return user_id

def create_user_vector(user_name):
    '''
    returns a 1d array of the ratings of one user
    unrated boardgames = 0
    '''
    user = create_user_ratings(user_name)
    vector_length = ratings['boardgame_id'].max()
    vector = np.repeat(0, vector_length+1)
    vector[user['boardgame_id']] = user['ratings']
    return vector

def create_user_ratings(user_name):
    '''
    returns a dataframe with rated boardgames for a specified user
    '''
    user_id = lookup_user_id(user_name)
    user = ratings[ratings['user_id']==user_id]
    return user

def values_to_list(df, column_name):
    categories = []
    for i in df[df[column_name].notna()].iterrows():
        categories = categories + i[1][column_name].split(', ')
    categories = list(dict.fromkeys(categories))
    categories.sort()
    return categories


def ohe_user_boardgames(user_name, column, weight=False):
    '''
    returns a one-hot-encoded matrix of parameters in column of games played by user
    if weight = True, the encoding gets weighted by the rating
    '''
    games_ohe={}
    user_id = users[users['user_name']==user_name]['user_id'].tolist()[0]
    user_ratings = ratings[ratings['user_id']==user_id].set_index('boardgame_id')
    user_boardgames = boardgames_ext.loc[user_ratings.index]
    user_boardgames = user_boardgames[user_boardgames[column].notna()]
    user_categories = values_to_list(user_boardgames, column)    
    for i in user_boardgames.iterrows():
        game_vector = [0]*len(user_categories)
        for c in i[1][column].split(', '):
            index = user_categories.index(c)
            if weight == True:
                game_vector[index]=1 * user_ratings.loc[i[0]]['ratings']
            else: 
                game_vector[index]=1
        games_ohe[i[0]] = game_vector
    df = pd.DataFrame(games_ohe)
    df = df.transpose()
    df.columns = user_categories
    return df

# Nearest Neighbor Model

In [34]:
ratings = pd.read_csv('../data/ratings_cleaned.csv')

In [7]:
ratings.head()

,boardgame_id,ratings,user_id
0,30549,10.0,214
1,822,10.0,214
2,13,10.0,214
3,68448,10.0,214
4,36218,10.0,214


In [168]:
ratings.shape

(20484318, 3)

In [170]:
ratings['user_id'].max()

265862

## Create sparse User-Item-Matrix

In [35]:
user_item = csr_matrix((ratings['ratings'], (ratings['user_id'], ratings['boardgame_id'])))

In [9]:
#shape = (number of users, max boardgame_id)
user_item.shape

(265863, 340910)

## Create Model

In [10]:
model = NearestNeighbors(metric='cosine')

In [7]:
model.fit(user_item)

NearestNeighbors(metric='cosine')

In [8]:
with open('../models/knn_model_cosine.pickle', 'wb') as file:
    pickle.dump(model, file)

## Create User Vectors

### a. of an existing user

In [11]:
users = pd.read_csv('../data/users.csv')
users.head(20)

,user_name,num_ratings,user_id
0,Walt Mulder,8675,0
1,Doel,8599,1
2,TomVasel,7052,2
3,warta,6902,3
4,jmdsplotter,6210,4
5,leffe dubbel,6131,5
6,Nap16,6097,6
7,Hessu68,5689,7
8,loopoocat,5663,8
9,oldgoat3769967,5299,9


In [12]:
user_id = users[users['user_name']=='Ser0']['user_id'].tolist()[0]
user_id

40701

In [13]:
user = ratings[ratings['user_id']==user_id]
user

,boardgame_id,ratings,user_id
13495904,30549,5.0,40701
13495905,68448,8.0,40701
13495906,36218,6.0,40701
13495907,178900,7.0,40701
13495908,167791,9.0,40701
...,...,...,...
13496026,303734,5.0,40701
13496027,42636,6.0,40701
13496028,165984,6.0,40701
13496029,135213,9.0,40701


In [14]:
user_boardgames = user['boardgame_id']

## Create User Vector

In [15]:
vector_length = ratings['boardgame_id'].max()
vector = np.repeat(0, vector_length+1)

In [16]:
vector[user['boardgame_id']] = user['ratings']

In [17]:
vector.shape

(340910,)

## Find Neighbors

In [16]:
distances, neighbor_ids = model.kneighbors([vector], n_neighbors=10)

In [17]:
neighbor_ids

array([[ 40701,  88695,  73514, 221205,  74607, 163111,  78724, 127072,
        123549, 161858]])

In [18]:
distances

array([[0.        , 0.67720592, 0.71442184, 0.72092662, 0.72340518,
        0.72377673, 0.72456408, 0.72643922, 0.72915679, 0.72969398]])

In [19]:
neighbor_filter = ratings['user_id'].isin(neighbor_ids[0][1:])
ratings[neighbor_filter]

,boardgame_id,ratings,user_id
6004886,30549,8.0,73514
6004887,822,8.0,73514
6004888,13,6.5,73514
6004889,68448,6.0,73514
6004890,36218,7.5,73514
...,...,...,...
19160361,281259,9.0,161858
19160362,251247,9.0,161858
19160363,184267,9.0,161858
19160364,283155,9.0,161858


In [20]:
neighbor_taste = ratings[neighbor_filter].groupby('boardgame_id').mean()['ratings'].sort_values(ascending=False)
neighbor_taste

boardgame_id
188       10.0
463       10.0
246900    10.0
3076      10.0
242277    10.0
          ... 
320        5.0
271264     5.0
180263     5.0
223779     3.0
2921       1.0
Name: ratings, Length: 220, dtype: float64

In [21]:
played_filter = ~neighbor_taste.index.isin(user_boardgames)
played_filter

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True, False, False, False,  True,  True,  True, False,  True,
        True, False,  True,  True, False, False, False, False, False,
        True, False,  True, False,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True, False, False,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True, False,  True, False,  True, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False,  True, False,  True,  True,  True, False,
       False,  True,

In [22]:
neighbor_taste[played_filter].index

Int64Index([   188,    463, 246900,   3076, 242277, 269385, 201808, 184267,
            126163, 232414,
            ...
              2719,  40398, 234877, 260300,   2223, 205896, 271264, 180263,
            223779,   2921],
           dtype='int64', name='boardgame_id', length=169)

## Find Top Categories, Mechanics etc.

In [140]:
boardgames = pd.read_csv('../data/boardgames.csv')
boardgames.shape

(45000, 5)

In [141]:
boardgames.head()

,rank,name,id,links,num_voters
0,104.0,Pandemic,30549,/boardgame/30549/pandemic,104891
1,184.0,Carcassonne,822,/boardgame/822/carcassonne,104530
2,408.0,Catan,13,/boardgame/13/catan,103960
3,71.0,7 Wonders,68448,/boardgame/68448/7-wonders,86475
4,101.0,Dominion,36218,/boardgame/36218/dominion,79469


In [23]:
boardgames_ext = pd.read_csv('../data/boardgames_extend_backup.csv')
boardgames_ext.shape

(45000, 12)

In [24]:
boardgames_ext.head()

,id,rank,name,links,num_voters,categories,mechanics,family,expansions,integrations,designers,publishers
0,30549,104.0,Pandemic,/boardgame/30549/pandemic,104891,Medical,"Action Points, Cooperative Game, Hand Manageme...","Components: Map (Global Scale), Components: Mu...",Pandemic: Gen Con 2016 Promos – Z-Force Team M...,NaN,Matt Leacock,"Z-Man Games, Inc., (Unknown), Albi, Asmodee, A..."
1,822,184.0,Carcassonne,/boardgame/822/carcassonne,104530,"City Building, Medieval, Territory Building","Area Majority / Influence, Map Addition, Tile ...","Cities: Carcassonne (France), Components: Meep...","20 Jahre Darmstadt Spielt, Apothecaries (fan e...",Carcassonne: Wheel of Fortune,Klaus-Jürgen Wrede,"Hans im Glück, 999 Games, Albi, Bard Centrum G..."
2,13,408.0,Catan,/boardgame/13/catan,103960,"Economic, Negotiation","Dice Rolling, Hexagon Grid, Income, Modular Bo...","Animals: Sheep, Components: Hexagonal Tiles, C...","20 Jahre Darmstadt Spielt, Brettspiel Adventsk...",NaN,Klaus Teuber,"KOSMOS, 999 Games, Albi, Astrel Games, Bergsal..."
3,68448,71.0,7 Wonders,/boardgame/68448/7-wonders,86475,"Ancient, Card Game, City Building, Civilizatio...","Drafting, Hand Management, Set Collection, Sim...","Digital Implementations: Board Game Arena, Gam...","7 Wonders: Armada, 7 Wonders: Babel, 7 Wonders...",NaN,Antoine Bauza,"Repos Production, ADC Blackfire Entertainment,..."
4,36218,101.0,Dominion,/boardgame/36218/dominion,79469,"Card Game, Medieval","Deck, Bag, and Pool Building, Delayed Purchase...","Crowdfunding: Wspieram, Game: Dominion, Misc: ...","Ancient Times (fan expansion for Dominion), An...",Dominion: Intrigue,Donald X. Vaccarino,"Rio Grande Games, 999 Games, Albi, Bard Centru..."


In [146]:
boardgames_ext['id'] = boardgames['id']

In [147]:
boardgames_ext.head()

,rank,name,links,num_voters,categories,mechanics,family,expansions,integrations,designers,publishers,id
0,104.0,Pandemic,/boardgame/30549/pandemic,104891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30549
1,184.0,Carcassonne,/boardgame/822/carcassonne,104530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,822
2,408.0,Catan,/boardgame/13/catan,103960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
3,71.0,7 Wonders,/boardgame/68448/7-wonders,86475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68448
4,101.0,Dominion,/boardgame/36218/dominion,79469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36218


In [148]:
boardgames_ext = boardgames_ext.set_index('id')

In [149]:
boardgames_ext['categories'].value_counts()

Abstract Strategy                                                                                           4
Card Game                                                                                                   4
Economic                                                                                                    3
Adventure, Exploration, Fantasy, Fighting, Miniatures                                                       3
Card Game, Number                                                                                           3
                                                                                                           ..
Fantasy, Puzzle                                                                                             1
Deduction, Fighting, Nautical, Real-time                                                                    1
Card Game, City Building, Economic                                                                          1
Dice, Econ

In [156]:
def values_to_list(df, column_name):
    categories = []
    for i in df[df[column_name].notna()].iterrows():
        categories = categories + i[1][column_name].split(', ')
    categories = list(dict.fromkeys(categories))
    categories.sort()
    return categories
        

In [276]:
def ohe_user_boardgames(user_name, column, weight=True):
    games_ohe={}
    user_id = users[users['user_name']==user_name]['user_id'].tolist()[0]
    user_ratings = ratings[ratings['user_id']==user_id].set_index('boardgame_id')
    user_boardgames = boardgames_ext.loc[user_ratings.index]
    user_boardgames = user_boardgames[user_boardgames[column].notna()]
    user_categories = values_to_list(user_boardgames, column)    
    for i in user_boardgames.iterrows():
        game_vector = [0]*len(user_categories)
        for c in i[1][column].split(', '):
            index = user_categories.index(c)
            if weight == True:
                game_vector[index]=1 * user_ratings.loc[i[0]]['ratings']
            else: 
                game_vector[index]=1
        games_ohe[i[0]] = game_vector
    df = pd.DataFrame(games_ohe)
    df = df.transpose()
    df.columns = user_categories
    return df

In [257]:
def ohe_user_boardgames(user_name, column, weight=True):
    games_ohe={}
    user_id = users[users['user_name']==user_name]['user_id'].tolist()[0]
    user_ratings = ratings[ratings['user_id']==user_id].set_index('boardgame_id')
    user_boardgames = boardgames_ext.loc[user_ratings.index]
    user_boardgames = user_boardgames[user_boardgames[column].notna()]
    user_categories = values_to_list(user_boardgames, column)    
    for i in user_boardgames.iterrows():
        game_vector = [0]*len(user_categories)
        for c in i[1][column].split(', '):
            index = user_categories.index(c)
            if weight == True:
                game_vector[index]=1 * user_ratings.loc[i[0]]['ratings']
            else: 
                game_vector[index]=1
        games_ohe[i[0]] = game_vector
    df = pd.DataFrame(games_ohe)
    df = df.transpose()
    df.columns = user_categories
    return df

In [266]:
sero = ohe_user_boardgames('Ser0', 'mechanics', weight=True)['Hand Management']

In [268]:
sero['boardgame_name'] = boardgames_ext.loc[sero.index]['name']

In [264]:
users_top_mechanics = ohe_user_boardgames('Ser0', 'mechanics').sum().sort_values(ascending=False)[:5].index.tolist()
users_top_mechanics

['Hand Management',
 'Card Drafting',
 'Solo / Solitaire Game',
 'Dice Rolling',
 'Variable Set-up']

# NMF Model

In [253]:
from sklearn.decomposition import NMF

In [254]:
ratings.shape

(20484318, 3)

In [255]:
user_item.shape

(265863, 340910)

In [256]:
user_item

<265863x340910 sparse matrix of type '<class 'numpy.float64'>'
	with 20321657 stored elements in Compressed Sparse Row format>

In [257]:
nmf = NMF(n_components=50, max_iter=600, init='nndsvd', alpha=0.01)

In [96]:
#nmf.fit(user_item)

NMF(alpha=0.01, init='nndsvd', max_iter=600, n_components=50)

In [248]:
#write model
#with open('../models/nmf_50.pickle', 'wb') as file:
#    pickle.dump(nmf, file)

In [261]:
#read model
with open('../models/nmf_50.pickle', 'rb') as file:
    nmf = pickle.load(file)

In [262]:
vector.shape

(340910,)

In [263]:
nmf.reconstruction_err_

28435.618386504528

In [264]:
Q = nmf.components_

In [265]:
P = nmf.transform([vector])

In [266]:
Q.shape, P.shape

((50, 340910), (1, 50))

In [267]:
predictions = np.dot(P, Q)
predictions

array([[0.        , 0.03044573, 0.01210105, ..., 0.        , 0.        ,
        0.00164359]])

In [268]:
predictions.shape

(1, 340910)

In [269]:
pseudo_ids = list(range(0,340910))

In [270]:
df = pd.DataFrame(predictions, columns=pseudo_ids)

In [271]:
recommendations_all = df.T.sort_values(0, ascending=False)
recommendations_all = recommendations_all.reset_index()
recommendations_all.columns = ['pseudo_id', 'pred_rating']
recommendations_all.head()

,pseudo_id,pred_rating
0,266192,11.776423
1,167791,8.348095
2,68448,7.536493
3,178900,7.263934
4,173346,6.830049


In [272]:
boardgames = pd.read_csv('../data/boardgames_extend.csv')
boardgames.head()

,Unnamed: 0,id,rank,name,links,num_voters,categories,mechanics,family,expansions,integrations,designers,publishers
0,0,30549,104.0,Pandemic,/boardgame/30549/pandemic,104891,Medical,"Action Points, Cooperative Game, Hand Manageme...","Components: Map (Global Scale), Components: Mu...",Pandemic: Gen Con 2016 Promos – Z-Force Team M...,NaN,Matt Leacock,"Z-Man Games, Inc., (Unknown), Albi, Asmodee, A..."
1,1,822,184.0,Carcassonne,/boardgame/822/carcassonne,104530,"City Building, Medieval, Territory Building","Area Majority / Influence, Map Addition, Tile ...","Cities: Carcassonne (France), Components: Meep...","20 Jahre Darmstadt Spielt, Apothecaries (fan e...",Carcassonne: Wheel of Fortune,Klaus-Jürgen Wrede,"Hans im Glück, 999 Games, Albi, Bard Centrum G..."
2,2,13,408.0,Catan,/boardgame/13/catan,103960,"Economic, Negotiation","Dice Rolling, Hexagon Grid, Income, Modular Bo...","Animals: Sheep, Components: Hexagonal Tiles, C...","20 Jahre Darmstadt Spielt, Brettspiel Adventsk...",NaN,Klaus Teuber,"KOSMOS, 999 Games, Albi, Astrel Games, Bergsal..."
3,3,68448,71.0,7 Wonders,/boardgame/68448/7-wonders,86475,"Ancient, Card Game, City Building, Civilizatio...","Drafting, Hand Management, Set Collection, Sim...","Digital Implementations: Board Game Arena, Gam...","7 Wonders: Armada, 7 Wonders: Babel, 7 Wonders...",NaN,Antoine Bauza,"Repos Production, ADC Blackfire Entertainment,..."
4,4,36218,101.0,Dominion,/boardgame/36218/dominion,79469,"Card Game, Medieval","Deck, Bag, and Pool Building, Delayed Purchase...","Crowdfunding: Wspieram, Game: Dominion, Misc: ...","Ancient Times (fan expansion for Dominion), An...",Dominion: Intrigue,Donald X. Vaccarino,"Rio Grande Games, 999 Games, Albi, Bard Centru..."


In [273]:
boardgames[['id','name']].head()

,id,name
0,30549,Pandemic
1,822,Carcassonne
2,13,Catan
3,68448,7 Wonders
4,36218,Dominion


In [274]:
prediction_df = pd.merge(boardgames[['id','name']], recommendations_all, left_on='id', right_on='pseudo_id', how='left')
prediction_df = prediction_df.set_index('id')
prediction_df.head()

,name,pseudo_id,pred_rating
id,,,
30549,Pandemic,30549,4.749527
822,Carcassonne,822,0.273716
13,Catan,13,0.024271
68448,7 Wonders,68448,7.536493
36218,Dominion,36218,5.316494


In [275]:
played_filter = ~prediction_df.index.isin(user_boardgames)
prediction_df.loc[played_filter].sort_values('pred_rating', ascending=False).head(20)
                                          

,name,pseudo_id,pred_rating
id,,,
40834,Dominion: Intrigue,40834,4.542982
183394,Viticulture Essential Edition,183394,3.879003
284083,The Crew: The Quest for Planet Nine,284083,3.150883
163412,Patchwork,163412,2.957857
236457,Architects of the West Kingdom,236457,2.788553
201808,Clank!: A Deck-Building Adventure,201808,2.371002
286096,Tapestry,286096,2.227416
205637,Arkham Horror: The Card Game,205637,2.222949
290448,Wingspan: European Expansion,290448,2.158668


## Combined kNN and NMF

In [151]:
P = nmf.transform(user_item)

In [239]:
# open a binary file in write mode
file = open('../models/P_50', 'wb')
# save array to the file
np.save(file, P)
# close the file
file.close

<function BufferedWriter.close>

In [167]:
P.shape

(265863, 50)

In [152]:
knn = NearestNeighbors(metric='cosine')
knn.fit(P)

NearestNeighbors(metric='cosine')

In [117]:
Q = nmf.components_

In [156]:
vector_transformed = nmf.transform([vector])
vector_transformed.shape

(1, 50)

In [157]:
vector_transformed = vector_transformed.T.reshape(50,)
vector_transformed.shape

(50,)

In [160]:
distances, neighbor_ids = knn.kneighbors([vector_transformed], n_neighbors=20)

In [161]:
neighbor_ids, distances

(array([[ 40701,  59834,  68546,  71553,  16639,  99654, 117602,  99865,
          64819,  95042,  42546,  73352,  71047, 179114,  19229,  14996,
          69353,  70900, 134807,  68639]]),
 array([[7.78617171e-11, 1.73424816e-01, 1.83562204e-01, 1.87554107e-01,
         1.88046754e-01, 1.95083727e-01, 1.95854162e-01, 1.98296969e-01,
         1.98848289e-01, 2.00000381e-01, 2.00449736e-01, 2.05966436e-01,
         2.05995836e-01, 2.06924451e-01, 2.08597866e-01, 2.09947744e-01,
         2.10096306e-01, 2.11213426e-01, 2.11715319e-01, 2.14067426e-01]]))

In [251]:
test = [188920, 199792, 245654, 167355, 277700, 306882, 246228, 224517, 205059, 63268, 217372, 200632, 167791, 284108, 244521, 242639, 272533, 232219, 135213, 73664, 266192, 120677, 181304, 311193, 218603, 68448, 194594, 173346, 180974, 247763, 318084, 250458, 264220, 291859, 283155, 318977, 244608, 230914, 40692, 178900, 237251, 91536, 263918, 45315, 192661, 162886, 1465, 2655, 283294, 251247, 206915, 204583, 214293, 264055, 207830, 299169, 235375, 155203, 169786, 48979, 193214, 216465, 284435, 299946, 303733, 172225, 202477, 21790, 314530, 258104, 128, 160477, 295486, 202977, 51811, 147020, 287938, 277927, 36218, 31260, 320, 237182, 147151, 239188, 66690, 4095, 217085, 227072, 31920, 121073, 274533, 10686, 282171, 262208, 260757, 6657, 181161, 39801, 305985, 305986, 3119, 232417, 300129, 288010, 63052, 18057, 42636, 165984, 50381, 66098, 185374, 98778, 209778, 206931, 90850, 256438, 173090, 143884, 30549, 181523, 303734, 15987, 305984, 1748, 204466, 122522, 50912]

In [252]:
user_ratings = create_user_ratings('Ser0')
user_ratings[user_ratings['boardgame_id'].isin(test)]

,boardgame_id,ratings,user_id
13495904,30549,5.0,40701
13495905,68448,8.0,40701
13495906,36218,6.0,40701
13495907,178900,7.0,40701
13495908,167791,9.0,40701
...,...,...,...
13496026,303734,5.0,40701
13496027,42636,6.0,40701
13496028,165984,6.0,40701
13496029,135213,9.0,40701


In [229]:
played_by_user = user_ratings['boardgame_id']
played_by_user

13495904     30549
13495905     68448
13495906     36218
13495907    178900
13495908    167791
             ...  
13496026    303734
13496027     42636
13496028    165984
13496029    135213
13496030    155203
Name: boardgame_id, Length: 127, dtype: int64

In [231]:
neighbor_ratings = ratings[ratings['user_id'].isin(neighbor_ids[0][1:])]
neighbor_ratings = neighbor_ratings[~neighbor_ratings['boardgame_id'].isin(played_by_user)]
neighbor_ratings.head()

,boardgame_id,ratings,user_id
610879,9209,7.0,68639
610883,148228,8.5,68639
610885,39856,6.0,68639
610887,34635,7.0,68639
610888,161936,10.0,68639


In [232]:
value_counts = pd.DataFrame(neighbor_ratings['boardgame_id'].value_counts())
value_counts.columns = ['count']
value_counts.head()

,count
183394,12
230802,11
40834,10
822,9
290448,9


In [233]:
frequently_played = value_counts[value_counts['count']>=5].index

In [234]:
recommendations = neighbor_ratings[neighbor_ratings['boardgame_id'].isin(frequently_played)].groupby('boardgame_id').mean().sort_values('ratings', ascending=False).head(15).index
recommendations

Int64Index([247030, 202174, 193738, 170216, 262712, 281259, 236457, 164928,
            121921, 183394, 125403, 254640, 290448, 170042, 233867],
           dtype='int64', name='boardgame_id')

In [243]:
boardgames = boardgames.set_index('id')

In [246]:
boardgames.loc[recommendations]['name']

boardgame_id
247030                          Terraforming Mars: Prelude
202174              Viticulture: Tuscany Essential Edition
193738                                 Great Western Trail
170216                                          Blood Rage
262712                                          Res Arcana
281259                                    The Isle of Cats
236457                      Architects of the West Kingdom
164928                                             Orléans
121921    Robinson Crusoe: Adventures on the Cursed Island
183394                       Viticulture Essential Edition
125403                                 Dominion: Dark Ages
254640                                            Just One
290448                        Wingspan: European Expansion
170042                            Raiders of the North Sea
233867                                       Welcome To...
Name: name, dtype: object

In [240]:
# open the file in read binary mode
file = open('../models/P_50', "rb")
#read the file to numpy array
Pl = np.load(file)

In [241]:
Pl

array([[0.00000000e+00, 2.66575675e+00, 2.08957898e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.27756620e-01],
       [0.00000000e+00, 4.10540895e+00, 2.70876590e+00, ...,
        1.57877620e-01, 3.46646538e-02, 7.18247691e-02],
       [1.42947393e-01, 2.37814182e+00, 1.53039253e+00, ...,
        7.48546486e-01, 0.00000000e+00, 2.76253613e-02],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.39125755e-03, 0.00000000e+00, ...,
        1.79553852e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.15435525e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [247]:
with open('../models/nmf_50.pickle', 'wb') as file:
    pickle.dump(model, file)